Importing and installing dependencies...

Upon research, I found a favourable approach to further pre-training was to perform further within-task or in-domain pretraining from the existing BERT model checkpoint.

I've already collected around 150,000 tweets that are categorized as either hate speech, offensive or benign so this roughly satisfies the requirement that the pretraining data is within-task or in-domain.

However there are far more tweet databases that can be used to further-pretrain my BERT model. There is a wealth of unsupervised tweet datasets online that I can make use of, these datasets often do not come in text form but rather the tweets are represented by their tweet IDs. Below I will describe the tweet datasets and retrieve their associated text using the tweepy module.

# Retrieving more pre-training data from unsupervised tweet datasets
Using as wide a variety of sources as we can will increase the learning of our further-pretrained model. My goal when sourcing these datasets was to try and find tweet datasets which are largely user-generated, as hate speech online largely comes from user generated content.

if I could find it, I would use tweet datasets likely to have aggressive or abusive content as well as possibly containing sexist and racial slurs. Even having some tweets just talking about racial, sex or homophobic issues would be beneficial for my model to be trained on as it may use the language associated with these issues.

The following tweets can all be retireved by ID and they come from the following sources:


*   <b>#UniteTheRight tweet database</b> - The Unite the Right rally (also known as the Charlottesville rally) was a protest in Charlottesville, Virginia, United States from August 11–12, 2017, to oppose the removal of a statue of Robert E. Lee in Emancipation Park, which itself was renamed from Lee Park two months earlier. Protesters included white supremacists, white nationalists, neo-Confederates, neo-Nazis, and militias. This dataset contains 200,113 tweet ids collected with the #unitetheright hashtag. The time ranges for the tweets are from 2017-08-04 11:44:12 to 2017-08-15 16:03:30 GMT.

*   <b>Bill 10 Twitter IDs</b> - A list of 24876 Twitter IDs for tweets harvested between Nov. 28 and Dec. 6 2014 containing the hashtag #bill10. Bill 10 in the Alberta legislature would have given public and Catholic school boards the right to refuse student requests to form gay-straight alliances in schools. Under intense public interest it was withdrawn by the Conservative government.

*   <b>BLMKidnapping</b> - These 136,990 tweet ids represent reaction to a Facebook Live video that was posted on January 3rd, 2017, showing four African American men violently attacking a white, mentally disabled man. The tweets were collected on 01/05/2017. After the video surfaced, the Twitter hashtag, #BLMkidnapping, was created and used to incorrectly attribute the violent attack to members of the Black Lives Matter movement. Police in Chicago, where the attack took place, have found no evidence the attack has any connection to the Black Lives Matter movement. This link is to a CNN story documenting the police denial of Black Lives Matter connection: http://www.cnn.com/2017/01/05/us/black-lives-matter-chicago-facebook-live-beating/index.html

*   <b>#Charlottesville</b> - On Friday, August 11th, 2017 large groups of racist white nationalists carrying torches marched on the University of Virginia campus in Charlottesville, VA as an intimidation tactic against proponents for the removal of confederate statues of Robert E. Lee. The Friday evening march was held ahead of a much larger racist white nationalist rally in the center of Charlottesville planned for Saturday, August 12th, 2017. This dataset includes 100,000 tweet ids and it includes tweets posted from 01:13:56 - 7:11:36 EDT on August 12.

*   <b>Replies to Ocasio-Cortez Tweets</b> - Replies to senator Rep. Alexandria Ocasio-Cortez’s Tweets and Retweets in March 2019. Whilst many tweets to Ms. Ocasio-Cortez may be glowing praise, I'm counting on a sizable portion of the tweets directed at her to be abusive and perhaps sexist and racist - I feel like this is a good enough hypothesis, owing to how toxic america's political climate has become.


Importing and Installing Dependencies ... 

In [1]:
import os
import pandas as pd
import re
import json
import tweepy

#Below is to authenticate google bucket access for local machines
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= "./storageCreds.json"
from google.cloud import storage
storage_client = storage.Client()
buckets = list(storage_client.list_buckets())
print(buckets)

#Below is for google colab environment
"""from google.colab import auth
auth.authenticate_user()
!gcloud config set project 'my-project-csc3002'"""

!pip install gcsfs

[<Bucket: csc3002>]


**Combining tweet ID dataframes**

In [2]:
#UniteTheRight
utr = 'gs://csc3002/pretrain_data/UniteTheRight.txt'
utr = pd.read_csv(utr, sep=',',  index_col = False, encoding = 'utf-8', header = None, names = ['id'])
print("#UniteTheRight tweet database contains", len(utr.index), "tweet IDs")

#Bill 10
b10 = 'gs://csc3002/pretrain_data/bill10tweets.txt'
b10 = pd.read_csv(b10, sep=',',  index_col = False, encoding = 'utf-8', header = None, names = ['id'])
print("\nBill 10 tweet database contains", len(b10.index), "tweet IDs")

#BlmKidnapping
blm = 'gs://csc3002/pretrain_data/blmkidnapping_tweet_ids_v1.txt'
blm = pd.read_csv(blm, sep=',',  index_col = False, encoding = 'utf-8', header = None, names = ['id'])
print("\n#blmkidnapping tweet database contains", len(blm.index), "tweet IDs")

#Ocasio-Cortez Replies
aoc = 'gs://csc3002/pretrain_data/ocasio_cortez_replies.csv'
aoc = pd.read_csv(aoc, sep=',',  index_col = False, encoding = 'utf-8', header = 0, names = ['id'])
print("\nAOC tweet database contains", len(aoc.index), "tweet IDs")



#UniteTheRight tweet database contains 200113 tweet IDs

Bill 10 tweet database contains 24876 tweet IDs

#blmkidnapping tweet database contains 136990 tweet IDs

AOC tweet database contains 109201 tweet IDs


Now combining all of the tweet IDs into a single dataframe.

In [3]:
full = pd.concat([aoc, blm, b10, utr], axis =0)
full.drop_duplicates(subset='id',inplace =True ) #Important to drop duplicates

#Shuffle Data
full = full.sample(frac=1)
full.reset_index(drop = True, inplace = True)

print("There are", len(full.index), "tweet IDs in this dataset")
full.head()

There are 471180 tweet IDs in this dataset


,id
0,896436979776143361
1,896565219501232128
2,897079831506153472
3,895836384468054017
4,896198992710717445


Below is a tweepy method to obtain tweets via ID. Twitter API only allows us to extract tweets 100 at a time, as there are rate limits - therfore, we must set the wait_on_rate_limit parameter to True.

In [4]:
def lookup_tweets(tweet_IDs, api):
    
    full_tweets = []
    tweet_count = len(tweet_IDs)
    
    #Below code to monitor progress
    x = int(tweet_count/5)
    progress = {x: '20%', x*2: '40%', x*3: '60%', x*4: '80%'}
    j = 0 # Remove mentions of j if i works to monitor progress
    try:
        for i in range((tweet_count // 100) + 1):
            if j in list(progress.keys()):
                print(progress[j], "complete")

            # Catch the last group if it is less than 100 tweets
            end_loc = min((i + 1) * 100, tweet_count)

            full_tweets.extend(
                api.statuses_lookup(id_=tweet_IDs[i * 100:end_loc], map_ = True))  
            j = j+1
            
        return full_tweets
    except tweepy.TweepError as e:
        print("At Index:", j, "\n", e.reason)       
        #Recursive call to continue even with exception
        full_tweets + lookup_tweets(tweet_IDs[i+1:tweet_count], api)
                

#Google colab
"""from google.colab import drive
drive.mount('/content/drive')
with open('/content/drive/My Drive/twitter_credentials.json', "r") as f:
  creds = json.load(f)"""

#Local machine
with open('./twitter_credentials.json', "r") as f:
    creds = json.load(f)


auth = tweepy.OAuthHandler(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])
auth.set_access_token(creds['ACCESS_TOKEN'], creds['ACCESS_SECRET'])

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, \
                 retry_count=10, retry_delay=5, retry_errors=set([503])) # These last three params catch over-capacity error

Applying the custom tweepy function to this dataframe to retrieve the text content corresponding to each tweet ID, then wrangling all of the text data into a singular dataframe.

<b>The below cell may take a while to run.</b>

In [5]:
tweet_ids = list(full['id'])

#Below works as long as it's not a multiple of 100. takes a while
results = lookup_tweets(tweet_ids, api)
                                  
temp = json.dumps([status._json for status in results]) #create JSON
final = pd.read_json(temp, orient='records')
final = final[['id','text']]
pd.set_option('display.max_colwidth', -1)
print(final.info())
final.head()

Rate limit reached. Sleeping for: 177
Rate limit reached. Sleeping for: 137
Rate limit reached. Sleeping for: 207
Rate limit reached. Sleeping for: 29


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 471180 entries, 0 to 471179
Data columns (total 2 columns):
id      471180 non-null int64
text    235551 non-null object
dtypes: int64(1), object(1)
memory usage: 7.2+ MB
None


,id,text
0,1108908334445322241,@AOC Says the bartender...
1,1102941794491351040,@AOC @bungarsargon Right now all you are doing is hurting our own party with division. Why can you just acknowledge… https://t.co/OYpLKjt6bM
2,896445976617132032,NaN
3,1111591492567613440,@dustbusterz @hmcghee @AOC @MSNBC Flopped as in her pushing her green new deal. But nuance takes some thought
4,540340474972626944,NaN


It'll be interesting to see if the amount of NaNs is the result of an error in my code, the database or if it's just banned accounts

In [6]:
#Quick function to beautify the error message
def getExceptionMessage(msg):
    words = msg.split(' ')

    errorMsg = ""
    for index, word in enumerate(words):
        if index not in [0,1,2]:
            errorMsg = errorMsg + ' ' + word
    errorMsg = errorMsg.rstrip("\'}]")
    errorMsg = errorMsg.lstrip(" \'")

    return errorMsg

def getErrorDesc(df): 
    for row in range(0, len(df.index)):
        current = df.loc[row]
        if pd.isna(current.text) == True:  
            try:    
                twt = api.get_status(current.id)
            except tweepy.TweepError as err:
                df.loc[row, 'text'] = getExceptionMessage(err.reason)
    return df

#If I attempt this with the whole dataset it'll take ages,
#So I'll just randomly sample the dataset to get a rough idea
df = final.sample(1000)
df.reset_index(drop = True, inplace = True)
print(df.info())
df = getErrorDesc(df)
print("\n", df.text.value_counts()[1:10])
df.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
id      1000 non-null int64
text    515 non-null object
dtypes: int64(1), object(1)
memory usage: 15.8+ KB
None
User has been suspended.                                                                                                                         204
Sorry, you are not authorized to see this status.                                                                                                28 
RT @AynRandPaulRyan: David Duke in #Charlottesville saying this Nazi, #UniteTheRight fiasco "fulfills the promises of Donald Trump." \nhttps…    24 
RT @RealAlexRubi: Massive brawl breaks out, tiki torches thrown as #UniteTheRight reaches Jefferson monument in #Charlottesville. Chemicals…     7  
RT @RVAwonk: Here's former KKK leader David Duke explicitly stating that Trump motivated the white supremacist #UniteTheRight rally in #Cha…     7  
RT @PrisonPlanet: Tucker is the only n

,id,text
0,896630257977085952,User has been suspended.
1,896375834415697920,User has been suspended.
2,897199189775527939,User has been suspended.
3,816865867371679744,RT @MattWalshBlog: Will @deray and @ShaunKing apologize for fomenting the kind of racial hate that led to the #BLMKidnapping?
4,1103074785859297280,User has been suspended.
5,893978535081201668,User has been suspended.
6,1111105464744493056,"@AOC This is not infrastructure @AOC this is building maintenance. \n\nInfrastructure is roads, airports, bridges..… https://t.co/hMqbzbc0Z4"
7,896476851572310016,"RT @AynRandPaulRyan: David Duke in #Charlottesville saying this Nazi, #UniteTheRight fiasco ""fulfills the promises of Donald Trump."" \nhttps…"
8,1107334308451237888,@my_surreality @JoeySalads @AOC Lol this guy... https://t.co/SlsnzlerQc
9,540531308183572480,RT @mikesbloggity: If you want to send an email to your MLA about #Bill10. Visit: http://t.co/C7onNIA7HH It takes less than ten seconds.


Interesting that not only has our function above showcased the errors associated, but also it's shown that there are many duplicate tweets as the result of RTs. We can remove this through the drop duplicates method

The text must be preprocessed before it becomes it is exported to a csv, otherwise it throws an error. The error is caused by the function not being able to encode unicode characters such as emojis

In [7]:
#The contraction mapping below is not perfect. There are many ambigious contractions
#which are impossible to definitively resolve (e.g. he's - he has or he is).
#The mappings below are unambigious but they are mapped to the most likely contractions
#We specifically choose flashtext as our library of choice for text replacement purposes
#because of it’s execution speed.
contractions = { 
"ain't": "is not","aren't": "are not", "can't": "cannot", "can't've": "cannot have",
"'cause": "because","cause": "because","could've": "could have","couldn't": "could not",
"couldn't've": "could not have","didn't": "did not","doesn't": "does not",
"don't": "do not","hadn't": "had not","hadn't've": "had not have","hasn't": "has not",
"haven't": "have not", "he'd": "he would","he'd've": "he would have", "he'll": "he will",
"he'll've": "he will have", "he's": "he is", "how'd": "how did", "how'll": "how will",
"how's": "how is ", "I'd": "I would", "I'd've": "I would have", "I'll": "I will",
"I'll've": "I will have","I'm": "I am","I've": "I have","isn't": "is not",
"it'd": "it had", "it'd've": "it would have","it'll": "it will", "it'll've": "it will have",
"it's": "it is", "let's": "let us", "ma'am": "madam","mayn't": "may not",
"might've": "might have","mightn't": "might not","mightn't've": "might not have",
"must've": "must have", "mustn't": "must not","mustn't've": "must not have",
"needn't": "need not", "needn't've": "need not have","oughtn't": "ought not",
"oughtn't've": "ought not have", "shan't": "shall not","sha'n't": "shall not",
"shan't've": "shall not have","she'd": "she had","she'd've": "she would have",
"she'll": "she will", "she'll've": "she will have","she's": "she is",
"should've": "should have","shouldn't": "should not","shouldn't've": "should not have",
"so've": "so have","so's": "so is","that'd": "that would", "that'd've": "that would have",
"that's": "that is", "there'd": "there had","there'd've": "there would have",
"there's": "there is","they'd": "they had", "they'd've": "they would have",
"they'll": "they will", "they'll've": "they will have", "they're": "they are",
"they've": "they have", "to've": "to have","wasn't": "was not",
"we'd": "we would","we'd've": "we would have","we'll": "we will","we'll've": "we will have",
"we're": "we are","we've": "we have","weren't": "were not","what'll": "what will",
"what'll've": "what will have","what're": "what are","what's": "what is",
"what've": "what have","when's": "when is","when've": "when have","where'd": "where did",
"where's": "where has","where've": "where have","who'll": "who will",
"who'll've": "who will have","who's": "who is","who've": "who have","why's": "why is",
"why've": "why have","will've": "will have","won't": "will not","won't've": "will not have",
"would've": "would have","wouldn't": "would not", "wouldn't've": "would not have",
"y'all": "you all","y'all'd": "you all would","y'all'd've": "you all would have",
"y'all're": "you all are","y'all've": "you all have","you'd": "you would",
"you'd've": "you would have","you'll": "you will", "you'll've": "you will have",
"you're": "you are", "you've": "you have"}

In [8]:
def preprocess(text_string):
    """
    Accepts a text string and:
    1) Removes URLS
    2) lots of whitespace with one instance
    3) Removes mentions
    4) Uses the html.unescape() method to convert unicode to text counterpart
    5) Replace & with and
    6) Remove the fact the tweet is a retweet if it is - knowing the tweet is 
       a retweet does not help towards our classification task.
    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[#$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+:'
    mention_regex1 = '@[\w\-]+'
    RT_regex = '(RT|rt)[ ]*@[ ]*[\S]+'
    
    # Replaces urls with URL
    parsed_text = re.sub(giant_url_regex, '', text_string)
    parsed_text = re.sub('URL', '', parsed_text)
    
    # Remove the fact the tweet is a retweet. 
    # (we're only interested in the language of the tweet here)
    parsed_text = re.sub(RT_regex, ' ', parsed_text) 
    
    # Removes mentions as they're redundant information
    parsed_text = re.sub(mention_regex, '',  parsed_text)
    #including ones with semi-colons after - this seems to come up often
    parsed_text = re.sub(mention_regex1, '',  parsed_text)  

    #Remove unicode
    parsed_text = re.sub(r'[^\x00-\x7F]','', parsed_text) 
    parsed_text = re.sub(r'&#[0-9]+;', '', parsed_text)  

    # Convert unicode missed by regex to text
   #parsed_text = html.unescape(parsed_text)

    #Remove excess whitespace at the end
    parsed_text = re.sub(space_pattern, ' ', parsed_text) 
    
    #Set text to lowercase and strip
    parsed_text = parsed_text.lower()
    parsed_text = parsed_text.strip()
    
    return parsed_text

Now concatenating the charlottesville dataset which already has the tweets associated with each ID so no text retrieval function via tweepy is necessary

In [ ]:
nullvals = final.text.isna().sum()
print("Amount of tweets IDs not returning text:", nullvals)
per = (nullvals/len(final.index)) * 100
print(("Which is %.2f%% of the overall dataset") % (per))


In [12]:
final.dropna(inplace = True)
final.reset_index(drop=True, inplace = True)
final['text'] = final['text'].apply(preprocess)

#Interim save
final.to_csv('gs://csc3002/Raw_Data/idstext.csv', sep = ',', encoding='utf-8', \
                 index = False, header = True)
final.tail()

,id,text
235546,896201447838187520,happening now at uva. our people on the march. will you be at #unitetheright tomorrow?
235547,896835566624546816,jason kessler organized the #unitetheright rally. he deserves the shaming for organizing &amp; the violence it incited. https
235548,1102729934894653440,i 2nd that shout out!
235549,1103276581060005889,perhaps it is just a
235550,896247350808784896,"alt right #unitetheright woman tells #antifa counter-protester that he ""sounds like a n-----"" #charlottesville"


In [20]:
#Charlottesville - Contains full text in columns
charl = 'gs://csc3002/pretrain_data/charlottesville_aug15_sample.csv'
charl = pd.read_csv(charl, sep=',',  index_col = False, encoding = 'utf-8', header = 0,)
charl1 = 'gs://csc3002/pretrain_data/aug16_sample.csv'
charl1 = pd.read_csv(charl1, sep=',',  index_col = False, encoding = 'utf-8', header = 0,)
charl2 = 'gs://csc3002/pretrain_data/aug17_sample.csv'
charl2 = pd.read_csv(charl2, sep=',',  index_col = False, encoding = 'utf-8', header = 0,)
charl3 = 'gs://csc3002/pretrain_data/aug18_sample.csv'
charl3 = pd.read_csv(charl3, sep=',',  index_col = False, encoding = 'utf-8', header = 0,)

dfs = [charl[['id','full_text']], charl1[['id','full_text']], \
       charl2[['id','full_text']], charl3[['id','full_text']]]
       
charlot = pd.concat(dfs, axis = 0)
charlot.rename(columns = {'full_text':'text'}, inplace = True)
print("\n#Charlottesville tweet database contains", len(charlot.index), "tweet IDs")

print(charlot.info())
charlot.head()


#Charlottesville tweet database contains 200000 tweet IDs
<class 'pandas.core.frame.DataFrame'>
Int64Index: 200000 entries, 0 to 49999
Data columns (total 2 columns):
id      200000 non-null int64
text    200000 non-null object
dtypes: int64(1), object(1)
memory usage: 4.6+ MB
None


,id,text
0,897661668787982336,It's almost as if people are exactly who they say they are https://t.co/MnWFXZd9c3
1,897654901534228480,"@Slate Conservative media: Yes, Trump's response to Charlottesville was bad, but what about Obama? https://t.co/jjINXL5Qp0 via @slate"
2,897659748597870592,👀 https://t.co/qeyzYeblwu
3,897660496656179202,😂 😂 😂 Karma really isn't wasting time.. https://t.co/JYRqf6vlSX
4,897642311903055872,"After Charlottesville, Black Lives Matter Issues New Demand - https://t.co/Vuw3IvrhL2"


In [23]:
final1 = pd.concat([final, charlot], axis = 0)
final1 = final1.sample(frac=1) #shuffle
final1['text'] = final1['text'].apply(preprocess)
final1.reset_index(drop = True, inplace = True)
final1.to_csv('gs://csc3002/pretrain_data/full.csv', sep = ',', encoding='utf-8', \
                 index = False, header = True)
final1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435551 entries, 0 to 435550
Data columns (total 2 columns):
id      435551 non-null int64
text    435551 non-null object
dtypes: int64(1), object(1)
memory usage: 6.6+ MB
